In [ ]:
!pip install caer canaro

In [ ]:
import os
import canaro
import caer
import numpy as np
import cv2 as cv
import gc

In [ ]:
image_size = (100,100)
channels = 1
directory = r'../input/stanford-dogs-dataset/images/Images'

In [ ]:
char_dict = {}
for char in os.listdir(directory):
    char_dict[char] = len(os.listdir(os.path.join(directory, char)))
    
# Descending order
char_dict = caer.sort_dict(char_dict, descending = True)
char_dict

In [ ]:
'''
To get dog species only
'''
# t = []
# saved = [t[0][10:] for t in char_dict]
# saved

'''
To connect image dirs
'''
count = 0
chars = []

for i in char_dict:
    chars.append(i[0])
    count += 1
    if count >= 200:
        break
chars

In [ ]:
# Create the training data
training = caer.preprocess_from_dir(directory, chars, channels = channels, IMG_SIZE = image_size, isShuffle = True)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(35,35))
plt.imshow(training[0][0], cmap = 'gray')
plt.show()

In [ ]:
featureSet, labels = caer.sep_train(training, IMG_SIZE = image_size)

In [ ]:
from tensorflow.keras.utils import to_categorical

#Normalize the featureSet ==> (0,1) to get faster
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(chars))


In [ ]:
# Models going to train on the training data and test itself on the validation data
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio =0.2)

In [ ]:
# Deletion of the variables that not going to be using
del training
del featureSet
del labels

gc.collect()

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
'''
Image data generator
Essentially synthesize new images from already existing images to help introduce
some randomness to make it perform better
'''

datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size = BATCH_SIZE)

In [ ]:
# Create and compile the Model
model = canaro.models.createSimpsonsModel(IMG_SIZE = image_size, channels = channels, output_dim = len(chars), loss = 'binary_crossentropy',
                                    decay = 1e-7, learning_rate = 0.001, momentum = 0.9, nesterov = True)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
training = model.fit(train_gen, steps_per_epoch = len(x_train)//BATCH_SIZE, epochs = EPOCHS,
                    validation_data = (x_val, y_val), validation_steps = len(y_val // BATCH_SIZE), 
                    callbacks = callbacks_list)

In [ ]:
chars

In [ ]:
test_path = r'../input/stanford-dogs-dataset/images/Images/n02088364-beagle/n02088364_10354.jpg'

img = cv.imread(test_path)

plt.imshow(img, cmap = 'gray')
plt.show()

In [ ]:
def prepare(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, image_size)
    img = caer.reshape(img, image_size, channels = 1)
    return img

In [ ]:
prediction = model.predict(prepare(img))

In [ ]:
print(chars[np.argmax(prediction[0])])